In [ ]:
import pandas as pd
import numpy as np
from openai import OpenAI, AsyncOpenAI

In [ ]:
aclient = AsyncOpenAI(
    api_key="sk-or-vv-726f75d0b47b51237e227344f747fe3ca4df5ed0622275827505ca1ce8e00e00", # ваш ключ в VseGPT после регистрации
    base_url="https://api.vsegpt.ru/v1",
)

In [ ]:
topics = [
    # Рабочие темы
    "Вопросы по документации",
    "Ошибки и баги в продуктах",

    # Технические вопросы
    "Обновления ПО",
    "Проблемы с серверами и инфрой",
    "Настройка инструментов и сервисов",
    "Вопросы по коду и разработке",

    "Обучение и повышение квалификации",
    "Вопросы оплаты труда",

    # Командные коммуникации и неформальное общение
    "Поздравления с днями рождения и праздниками",
    "Досуг и хобби сотрудников",

    # Клиенты и партнеры
    "Запросы и обращения клиентов",
    "Обсуждение сложных клиентов",
    "Подготовка к встречам и презентациям",

    # Any random stuff
    "Блог",
    "Любой Форум"
] + ["Любой"]*5


prompt = """\
Ты — помощник, который генерирует сообщения для корпоративных чатов, блогов, новостей. \
Тебе нужно сгенерировать текст, соответствующее указанному настроению (sentiment) и заданного контекста.
Ниже могут быть представлены примеры случайных сообщений из чатов, гневных комментариев или сообщений в блог постах.
Текст должен быть похожим по **вайбам** на примеры, \
то есть иногда быть не формальным, быть в определенной раскладке, или содержать, знаки препинания или различные эмоции типа <3, если такие есть в примерах,\
но обязательно должен соотвествовать указанному sentiment. Также сообщения могут сильно отличаться по длине, ты должен тоже подстраиваться.
Не пиши слишком обобщенно, придумывай имена, места, даты, цифры или любую другую информацию, которая может появиться в сообщениях, для большего разнообразия.
Ниже есть несколько примеров такого вида:
- Sentiment: Настроение сообщения (например, нейтральное, позитивное, негативное).
- Text: Сам текст сообщения, комментария или блога.

### Примеры:
{examples}

Теперь твоя задача сгенерировать {n} новых сообщений с "{target_label}" sentiment. Убедись, что стиль и контекст совпадают с примерами.

Ответ должен быть в формате:
- <text 1>
- <text 2>
- ...
"""

In [ ]:
df = pd.read_excel('./a_with_labels.xlsx')
df = df[['Text', 'my-label']].rename(columns={'Text': 'text', 'my-label': 'label'})

In [ ]:
from cosette import *

In [ ]:
acli = Client('openai/gpt-4o-mini', aclient)

In [ ]:
import re


def parse_response(response):
    pattern = r"^- (.+)$"
    matches = re.findall(pattern, contents(response), re.MULTILINE)
    return matches


def format_prompt(prompt, rows, label: str, n_texts: int = 5):
    examples = "\n".join(
        [f"- Sentiment: {row['label']}\n- Text: {row['text']}" for _, row in rows.iterrows()]
    )
    pr = prompt.format(examples=examples, target_label=label, n=n_texts)
    return pr


def save_data(generated, fname='generated.xlsx'):
    flattened_data = []
    for sentiment, texts, _ in generated:
        for text in texts:
            flattened_data.append({"label": sentiment, "text": text})
    pd.DataFrame(flattened_data).to_excel(fname, index=None)

In [ ]:
import asyncio
from tqdm.auto import trange
import time

label_counts = df['label'].value_counts()
weights = 1.0 / label_counts[df['label']].values
weights /= weights.sum()
n_texts = 5
n_examples = 5

total = 10_000
fname = 'generated.xlsx'
sampled_counts = np.zeros(len(df), dtype=np.int64)


generated = []
prompted = []
for i in trange(total):
    try:
        sampled_df = df.sample(n=n_examples, weights=weights)
        label = sampled_df['label'].sample(1).iloc[0]
        pr = format_prompt(prompt, sampled_df, label, n_texts)

        start_time = time.time()

        res = parse_response(await acli([mk_msg(pr)]))
        # keep RPS=1
        elapsed_time = time.time() - start_time
        if elapsed_time < 1: await asyncio.sleep(1 - elapsed_time)

        generated.append((label, res, pr))
        prompted.append(sampled_df)
        if i and i % 30 == 1: save_data(generated, fname)
    except Exception as e: print(e)
    except (KeyboardInterrupt, asyncio.CancelledError): 
        save_data(generated, fname)
        break

  0%|          | 0/10000 [00:00<?, ?it/s]

[Errno 13] Permission denied: 'generated.xlsx'
[Errno 13] Permission denied: 'generated.xlsx'


## Paraphrase

In [ ]:
prompt = """\
Ты — помощник, который генерирует или перефразирует похожие сообщения для корпоративных чатов, блогов, новостей.

Теперь твоя задача сгенерировать {n} новых сообщений, которые похожи на текущее.
Похожее - значит, что у сгенерированного текста сохраняется настроение оригинального, сохраняется средняя длина, сохранятся посыл текста.
Все остальное может меняться, так как нужно получить большую вариативность.
Исходный текст:
{text}

Ответ должен быть в формате:
- <text 1>
- <text 2>
- ...
"""

In [ ]:
import asyncio
from tqdm.auto import tqdm
import time

n_texts = 5

fname = 'paraphrased.xlsx'

generated = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        pr = prompt.format(n=n_texts, text=row['text'])
        label = row['label']
        start_time = time.time()
        res = parse_response(await acli([mk_msg(pr)]))
        # keep RPS=1
        elapsed_time = time.time() - start_time
        if elapsed_time < 1: await asyncio.sleep(1 - elapsed_time)
        generated.append((label, res, pr))
        if i and i % 30 == 1: save_data(generated, fname)
    except Exception as e: print(e)
    except (KeyboardInterrupt, asyncio.CancelledError): 
        save_data(generated, fname)
        break

  0%|          | 0/508 [00:00<?, ?it/s]